# PHYS20762 Computational Physics
## Project 3: Monte Carlo Techniques
### Neutron Transport and Scattering Through a Shielding Layer

Author: David Phelan  
The University of Manchester  
Date: May 2025  

---

## 1. Introduction

In this project, we use Monte Carlo techniques to simulate the behaviour of thermal neutrons passing through various shielding materials. The aim is to model neutron scattering, absorption, and transmission processes within slabs of water, lead, and graphite.

By generating random numbers, random points in three dimensions, and implementing exponential distributions, we build up to a full neutron transport simulation. We will visualise random walks, quantify transmission, reflection, and absorption rates, and determine the characteristic attenuation lengths for different materials.

### 1.1 Objectives

- Verify the generation of uniform random numbers using `numpy.random.uniform()`.
- Generate and visualise random 3D points to assess distribution properties.
- Create and test an exponential random number generator.
- Produce isotropic random directions and isotropic steps with exponentially distributed lengths.
- Simulate neutron random walks through slabs of different materials.
- Quantify neutron absorption, reflection, and transmission rates as functions of slab thickness.
- Determine attenuation lengths from transmission data.
- Implement the Woodcock method to simulate transport through two adjacent materials.

Throughout the project, we aim to support the simulation with appropriate plots and discuss the numerical results, including any uncertainties.

---

## 2. Setup & Initialisation
We first set up the computational environment by:  
- Installing and importing essential libraries.  
- Ensuring all dependencies are available before proceeding.  

#### Imported Libraries
- `numpy` – for numerical computations and array handling.
- `scipy.optimize` – for curve fitting and optimisation tasks.
- `plotly.graph_objects` – for creating interactive 3D plots and visualisations.
- `collections.Counter` – for efficient counting of elements.

These libraries provide the necessary tools to perform random number generation, simulate neutron transport, visualise the results, and manage the computational workflow effectively.

In [ ]:
# Install required packages (only run this (once) if needed)
%pip install numpy scipy plotly

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from collections import Counter
from IPython.display import display, Markdown
import textwrap
import time

### Random Seed for Reproducibility
To ensure consistent and reproducible results across different runs of the notebook, we set a fixed random seed at the start of the script. This guarantees that each random number generated will follow the same sequence every time the notebook is executed (for example, if the marker uses "Run All" in Jupyter Notebook).

In [ ]:
np.random.seed(42)
start = time.time()

---

## 3. Simulation Classes

To manage the material properties and slab geometries systematically, we define two classes: `Material` and `Slab`.

### `Material` Class

The `Material` class encapsulates the physical properties needed to simulate neutron interactions, including:

- **Scattering cross-section** ($σ_s$)
- **Absorption cross-section** ($σ_a$)
- **Total cross-section** (sum of scattering and absorption)
- **Absorption probability**

It also includes a `construct_from_microscopic_properties` method to conveniently calculate macroscopic cross-sections based on:

- `density` $\rho$
- `molar_mass` $M$
- Microscopic cross-sections (given in barns, converted to ${cm}^2$)

This design improves clarity and avoids recalculating these properties throughout the notebook.

### `Slab` Class

The `Slab` class represents a slab of shielding material, characterised by:

- A `Material` object (e.g., water, lead, graphite)
- A `width` (thickness $L$ through which neutrons travel

By creating a separate `Slab` class, we make the simulation modular, allowing us to easily vary material type and slab thickness during experiments.

In [ ]:
class Material:
    BARN_TO_CM2 = 1e-24  # cm²
    AVOGADRO_CONSTANT = 6.02214076e23  # 1/mol

    def __init__(self, name, scattering_cross_section, absorption_cross_section):
        self.name = name
        self.scattering_cross_section = scattering_cross_section
        self.absorption_cross_section = absorption_cross_section
        self.total_cross_section = scattering_cross_section + absorption_cross_section
        self.absorption_probability = absorption_cross_section / self.total_cross_section

    @staticmethod
    def construct_from_microscopic_properties(name, sigma_a, sigma_s, density, molar_mass):
        number_density = Material.AVOGADRO_CONSTANT * density / molar_mass
        

        return Material(
            name,
            number_density * sigma_s * Material.BARN_TO_CM2,
            number_density * sigma_a * Material.BARN_TO_CM2
        )
    
class Slab:
    def __init__(self, material, width):
        self.material = material
        self.width = width

### Initialisation of Materials

We instantiate the following materials using their known physical properties:

In [ ]:
# Define materials with their properties
water = Material.construct_from_microscopic_properties(
    "Water", 0.6652, 103.0, 1.0, 18.0153)

lead = Material.construct_from_microscopic_properties(
    "Lead", 0.158, 11.221, 11.35, 207.2)

graphite = Material.construct_from_microscopic_properties(
    "Graphite", 0.0045, 4.74, 1.67, 12.011)

materials = [water, lead, graphite]

Each material's macroscopic properties are now readily accessible for subsequent neutron transport simulations.

Setting the figure dimensions (in number of pixels) to be used throughout the script.

In [ ]:
plot_width = 1920/2
plot_height = 1080/2

def print_markdown(markdown):
    display(Markdown(textwrap.dedent(markdown)))



---

## 4. Testing Random Number Generators

### 4.1 Uniform Random Number Generator (`numpy.random.uniform`)

We first test Python's built-in uniform random number generator, `numpy.random.uniform()`.  
In one dimension, a uniform random generator should produce a flat probability density between 0 and 1.  
In three dimensions, random (x, y, z) points should fill the space evenly without any clustering, streaks, or artefacts.  
We visualise the results using a histogram of random values and a 3D scatter plot.

In [ ]:
def test_random_number_generator():
    x = np.random.uniform(0, 1, 1000)
    y = np.random.uniform(0, 1, 1000)
    z = np.random.uniform(0, 1, 1000)

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Uniform)", "3D Scatter (Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Uniform 3D Points"
        ),
        row=1, col=2
    )

    fig.update_layout(
        width=1200,
        height=600
    )

    fig.show()

test_random_number_generator()

### 4.2 Biased Random Number Generator (`RandSSP`)

We next test a flawed random number generator, `RandSSP`, which is known to introduce bias into random samples.  
Such flaws can lead to visible structure in the generated points, such as banding or clustering, which should not occur for truly random distributions.  
We again visualise the results using a histogram of random values and a 3D scatter plot, and compare them to the output of `numpy.random.uniform()`.

Bad one:
watch random num gen video for info (phsyics) about why it's bad

In [ ]:
class RandSSP:
    def __init__(self, seed=123456789):
        self.m = 2**31
        self.a = 2**16 + 3
        self.c = 0
        self.x = seed

    def generate(self, p, q):
        r = np.zeros((p, q))
        for l in range(q):
            for k in range(p):
                self.x = (self.a * self.x + self.c) % self.m
                r[k, l] = self.x / self.m
        return r

In [ ]:
def test_random_number_generator():
    r = RandSSP().generate(3, 1500)
    x, y, z = r[0, :], r[1, :], r[2, :]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Non-Uniform)", "3D Scatter (Non-Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Non-Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Non-Uniform 3D Points"
        ),
        row=1, col=2
    )

    fig.update_layout(
        width=1200,
        height=600
    )

    fig.show()

test_random_number_generator()

The `RandSSP` generator produces clear evidence of bias:

- The histogram shows a non-uniform spread, with $y$-values mainly between 20 and 90.
- The 3D scatter plot reveals points aligned in planes within the $xy$-plane, highlighting structural artefacts.


---

## 5 Survival Function Fitting

In order to determine the attenuation length of neutrons passing through a shielding material, we analyse the survival function — the number of neutrons that have travelled at least a given distance without being absorbed.

One basic approach to constructing the survival function is to bin the neutron travel distances into discrete intervals and count the number of neutrons surviving within each bin. However, binning inherently introduces systematic error whenever a continuous distribution is discretised. Unless bins are infinitesimally small, each bin groups together neutrons that travelled different distances, distorting the true shape of the survival function. This distortion is particularly severe for exponential decay processes, which are "left-heavy" — most neutrons are absorbed at short distances, and relatively few survive to large distances. When bin midpoints are used, the majority of events occur to the left of the bin centre, leading to a systematic rightward bias: the binned survival function falsely suggests that neutrons survive slightly longer than they actually do. Alternative schemes such as left-edge binning can overcompensate, introducing a leftward bias. To avoid all such bias, we adopt a ranked method, sorting individual neutron travel distances and computing the number of survivors directly, without binning. This ensures that the survival function accurately reflects the underlying physics of neutron attenuation without distortion from binning artefacts.

Since each survival count is subject to Poisson statistics, the uncertainty in the number of surviving neutrons $ N(x)$ is approximately $\sqrt{N(x)}$. When fitting the logarithm of the survival function, the corresponding uncertainty in $\log(N(x))$ is $1/\sqrt{N(x)}$. Therefore, when performing a weighted linear regression of $\log(\text{survivors})$ against distance, we assign each point a weight proportional to $\sqrt{N(x)}$, ensuring that points with higher neutron counts (and thus lower relative noise) have greater influence on the fit.

On each survival function plot, we also display a horizontal dashed line at $N_0/e$, where $N_0$ is the initial number of neutrons. Theoretically, this line represents the expected number of surviving neutrons after travelling one attenuation length through the material, as given by the exponential decay law $N(x) = N_0 e^{-x/\lambda}$. The $x$ value of its inersection with the fitted function gives the attenuation
length of the material.

By fitting the survival function using this ranked, weighted approach, we obtain a highly accurate estimate of the material’s attenuation length while minimising systematic bias from data binning. This method is conducted for water:

The material’s absorption cross section is used while scattering is neglected at this stage. Scattering is more complex and requires a full random walk simulation, which is carried out later.

In [ ]:
def calculate_reduced_chi_square(observed, expected):

    observed = np.asarray(observed)
    expected = np.asarray(expected)

    chi_squared = np.sum((observed - expected) ** 2 / expected)
    dof = len(observed) - 2
    reduced_chi_squared = chi_squared / dof

    return reduced_chi_squared


In [ ]:
def produce_exponential_distribution(samples, material):
    uniform_random_numbers = np.random.uniform(0, 1, samples)
    exponential_random_numbers = - np.log(uniform_random_numbers) / material.absorption_cross_section

    sorted_data = np.sort(exponential_random_numbers)
    ranks = np.arange(len(sorted_data), 0, -1)

    x_ranked = sorted_data
    y_ranked = np.log(ranks)

    weights_ranked = np.sqrt(ranks)  

    fit_ranked, cov_ranked = np.polyfit(x_ranked, y_ranked, 1, w=weights_ranked, cov=True)
    slope_ranked = fit_ranked[0]
    slope_uncertainty_ranked = np.sqrt(cov_ranked[0][0])

    attenuation_length_ranked = -1 / slope_ranked
    uncertainty_ranked = slope_uncertainty_ranked / slope_ranked**2

    return x_ranked, ranks, fit_ranked, attenuation_length_ranked, uncertainty_ranked

In [ ]:
def plot_exponential_distribution(x_ranked, ranks, fit_ranked, attenuation_length_ranked, uncertainty_ranked, material, samples):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=x_ranked,
        y=ranks,
        mode='markers',
        name=f"Ranked survival ({material.name})"
    ))

    fitted_y = np.exp(fit_ranked[1] + fit_ranked[0] * x_ranked)
    fig.add_trace(go.Scatter(
        x=x_ranked,
        y=fitted_y,
        mode='lines',
        name=f"Exponential fit ({material.name})"
    ))

    samples_after_one_attenuation_length = samples / np.e

    line_trace = go.Scatter(
        x=[0, max(x_ranked)],
        y=[samples_after_one_attenuation_length, samples_after_one_attenuation_length],
        mode='lines',
        line=dict(color='red', dash='dash'),
        name=f"{material.name}: N₀/e Line"
    )

    fig.add_trace(line_trace)

    fig.add_annotation(
        x=max(x_ranked) * 0.8,
        y=samples_after_one_attenuation_length,
        text="N₀/e",
        showarrow=False,
        font=dict(size=12, color="red"),
        yshift=10
    )

    fig.update_layout(
        title=f"Ranked Survival Function and Weighted Fit for {material.name}",
        width=plot_width,
        height=plot_height,
        xaxis_title="Distance (cm)",
        yaxis_title="Number of neutrons surviving",
        legend=dict(x=0.68, y=0.98)
    )

    fig.show()

In [ ]:
def print_results():
    ranked_data = produce_exponential_distribution(samples=1000, material=water)
    plot_exponential_distribution(*ranked_data, material=water, samples=1000)

    data = [(water, ranked_data), *[(material, produce_exponential_distribution(samples=1000, material=material)) for material in [lead, graphite]]]
    markdown = "Material | Theoretical Attenuation Length (cm) | Simulated Attenuation Length (cm) | Uncertainty (cm) | $\chi_R^2$ \n"
    markdown += "--- | --- | --- | --- | --- \n"
    for material, ranked_data in data:
        attenuation_length_ranked, uncertainty_ranked = ranked_data[3], ranked_data[4]
        theoretical_attenuation_length = 1 / material.absorption_cross_section
        reduced_chi_square_ranked = calculate_reduced_chi_square(ranked_data[1], np.exp(ranked_data[2][1] + ranked_data[2][0] * ranked_data[0]))
        markdown += f"{material.name} | {theoretical_attenuation_length:.2f} | {attenuation_length_ranked:.2f} | {uncertainty_ranked:.2f} | {reduced_chi_square_ranked:.2f} \n"

    print_markdown(markdown)

print_results()

discuss results above.


---

## 6. Generating Isotropic Directions

### 6.1 Generating Isotropic Unit Vectors

To simulate random neutron directions uniformly distributed over a sphere, we generate isotropic unit vectors $\vec{r} = x\hat{i} + y\hat{j} + z\hat{k}$ satisfying $|\vec{r}| = 1$.  
We use spherical polar coordinates $(\theta, \phi)$ where:
- $\theta$ is the polar angle, drawn from $\cos\theta \sim \text{uniform}(-1, 1)$,
- $\phi$ is the azimuthal angle, drawn uniformly from $[0, 2\pi)$.

This ensures points are uniformly distributed over the sphere's surface, representing isotropic scattering accurately.

If we sampled $\theta$ uniformly from $[0, \pi]$, the resulting points would cluster near the poles due to the non-uniform area element in spherical coordinates. By sampling $\cos\theta$ uniformly instead, we correct for this bias and ensure an even distribution over the sphere’s surface.


In [ ]:
def generate_random_unit_vector():

    theta = np.arccos(np.random.uniform(-1, 1))
    phi = np.random.uniform(0, 2*np.pi)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    return x, y, z

## 6.1.1 Visualising the Isotropic Unit Vectors

To verify the uniformity of the generated directions, we visualise many unit vectors in 3D space.  
A truly isotropic distribution should appear as a uniform scatter of points across the surface of a sphere.

In [ ]:
def visualize_random_unit_vectors():
    x,y,z = zip(*[(x,y,z) for x,y,z in [generate_random_unit_vector() for _ in range(10000)]])

    fig6 = px.scatter_3d(
        x=x, y=y, z=z, color=z,
        labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
    )
    fig6.update_traces(marker=dict(size=2))  # Smaller points
    fig6.update_layout(
        title="3D Scatter Plot",
        width=plot_width,
        height=plot_height
    )
    fig6.show()

visualize_random_unit_vectors()

## 6.2 Simulating Random Walks in 3D Space

Using the isotropic unit vectors generated above, we now simulate random neutron paths through 3D space.  
At each step, a neutron travels in a random direction, with a step length sampled from an exponential distribution:

$$P(x) = \frac{1}{\lambda} e^{-x/\lambda}$$

where $\lambda$ is the mean free path determined by the material's macroscopic cross-section.  
This simulates the stochastic behaviour of neutrons undergoing scattering and absorption in matter.

In [ ]:
def random_uniform_path_length(cross_section):
    return -np.log(np.random.uniform(0, 1)) / cross_section

In [ ]:
def random_walk(cross_section):
    x, y, z = (0, 0, 0)
    yield (x, y, z)
    while True:
        step_size = random_uniform_path_length(cross_section)
        dx, dy, dz = generate_random_unit_vector()
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        yield (x, y, z)

## 6.2.1 Visualising Random Walks

We now simulate neutron random walks through different materials.  
Each trajectory represents the stochastic motion of a neutron, with direction sampled isotropically and step lengths sampled from the material's exponential attenuation law.

In [ ]:
def visualise_random_walk(materials):
    for material in materials:
        macroscopic_cross_section= material.absorption_cross_section
        
        positions_iterator = random_walk(macroscopic_cross_section)
        positions = [next(positions_iterator) for _ in range(100)]

        x, y, z = zip(*positions)
        scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', marker=dict(size=2))
        layout = go.Layout(
            title=f"Random Walk in {material.name}",
            scene=dict(
                xaxis=dict(title='X'),
                yaxis=dict(title='Y'),
                zaxis=dict(title='Z')
            )
        )
        fig = go.Figure(data=[scatter], layout=layout)
        fig.show()

visualise_random_walk(materials)

This is just random walks not factoring in abs or trans rights


# end of week 2

## 6.3 Visualising Neutron Random Walks in a Slab

We now simulate neutron random walks through a finite slab of material.  
Each slab is placed between two planes located at $x = 0$ and $x = L$, where $L$ is the slab thickness.  

Initially, neutrons are fired **perpendicularly into the slab** from the left boundary at $x = 0$, moving along the positive $x$-axis.  
Each neutron then undergoes a stochastic random walk within the slab:
- The **step size** along its path is drawn from an exponential distribution:
  
  $$P(x) = \frac{1}{\lambda} e^{-x/\lambda}$$
  
  where $\lambda$ is the mean free path based on the material's macroscopic cross-section.
- After each step, the neutron's new position is updated by moving along a randomly generated isotropic unit vector.

At each step, we determine whether the neutron:
- **Is absorbed** within the slab (random survival check against absorption probability),
- **Scatters** and continues walking within the slab,
- **Escapes** by crossing beyond the slab boundaries (either $x < 0$ or $x > L$).

The random walk terminates if the neutron is absorbed or escapes.  
We visualise individual neutron trajectories through the slab for different materials, illustrating the stochastic scattering and absorption behaviour in each case.

In [ ]:
def random_walk(slab):
    result = []
    x, y, z = (0, 0, 0)
    result.append((x, y, z))
    dx, dy, dz = (1, 0, 0)  # Initial direction
    while True:
        step_size = random_uniform_path_length(slab.material.total_cross_section)
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        if np.random.uniform(0, 1) < slab.material.absorption_probability:
            return "Absorbed", result
        elif x > slab.width:
            return "Transmitted", result
        elif x < 0:
            return "Reflected", result
        else:
            # Scattering occurs
            dx, dy, dz = generate_random_unit_vector()
        result.append((x, y, z))

In [ ]:
def visualise_random_walks(slab, number_of_neutrons):
    walks = [ random_walk(slab) for _ in range(number_of_neutrons) ]

    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    show_legend = {
        "Transmitted": True,
        "Reflected": True,
        "Absorbed": True
    }
    colours = {
        "Transmitted": "blue",
        "Reflected": "red",
        "Absorbed": "green"
    }
    fate_counts = Counter([walk[0] for walk in walks])

    def walk_to_scatter(walk):
        x, y, z = zip(*walk[1])
        showlegend = show_legend[walk[0]]
        show_legend[walk[0]] = False #only show the legend for the first walk of each type
        return go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=2), name=f'{walk[0]} ({fate_counts[walk[0]]})', line=dict(color=colours[walk[0]]), showlegend=showlegend)
    
    fig = go.Figure(data=[walk_to_scatter(walk) for walk in walks], layout=layout)
    fig.update_layout(
        title=f"Random Walk in {slab.material.name}",
        width=plot_width,
        height=plot_height
    )
    fig.show()

visualise_random_walks(Slab(water, 4), 30)


### 6.4 Quantifying Neutron Scattering, Absorption, and Transmission

We now quantify the outcomes of neutron interactions with the slab by simulating large numbers of random walks.

For a given slab thickness $L$, we fire $N$ neutrons perpendicularly into the slab at $x = 0$, each initially travelling along the positive $x$-direction. Each neutron undergoes a random walk based on the material's macroscopic cross-section, continuing until it is either:
- **Absorbed** within the slab,
- **Reflected** back through the entrance at $x < 0$,
- **Transmitted** through the far side at $x > L$.

We record the number of neutrons absorbed ($N_A$), reflected ($N_R$), and transmitted ($N_T$), and compute the corresponding rates:

$$\text{Absorption Rate} = \frac{N_A}{N}, \quad \text{Reflection Rate} = \frac{N_R}{N}, \quad \text{Transmission Rate} = \frac{N_T}{N}$$

The statistical uncertainty in each measured rate is estimated assuming binomial statistics:

$$\sigma = \sqrt{\frac{p(1-p)}{N}}$$

where $p$ is the measured probability of absorption, reflection, or transmission.

In [ ]:
def quantify_random_walk(slab, number_of_neutrons):

    walks = [random_walk(slab) for _ in range(number_of_neutrons)]
    fate_counts = Counter([walk[0] for walk in walks])
    for fate in fate_counts:
        rate = fate_counts[fate] / number_of_neutrons
        rate_unc = np.sqrt(rate * (1 - rate) / number_of_neutrons)
        yield fate, rate, rate_unc

def show_quantified_random_walk(slabs, number_of_neutrons):
    fig = go.Figure()
    for slab in slabs:
        fates = quantify_random_walk(slab, number_of_neutrons)
        fates = list(fates)
        fates = sorted(fates, key=lambda x: x[0])
        fates = [(fate[0], fate[1] * 100, fate[2] * 100) for fate in fates]
        fates = [(fate[0], fate[1], fate[2]) for fate in fates if fate[1] > 0.01]

        fig.add_trace(go.Bar(
            x=[fate[0] for fate in fates],
            y=[fate[1] for fate in fates],
            name=slab.material.name,
            error_y=dict(type='data', array=[fate[2] for fate in fates])
        ))

    fig.update_layout(
        title="Quantified Random Walk Results",
        xaxis_title="Fate",
        yaxis_title="Rate (%)",
        barmode='group',
        width=plot_width,
        height=plot_height,
        legend=dict(x=0.68, y=0.98),
        yaxis=dict(range=[0, 100])
    )
    fig.show()

show_quantified_random_walk([Slab(water, 10), Slab(graphite, 10), Slab(lead, 10)], 1000)

## 6.4.3 Variation of Neutron Interaction Rates with Slab Thickness

To analyse how neutron behaviour depends on material thickness, we repeat the random walk simulation for a range of slab thicknesses $L$.  
At each thickness, we compute the absorption, reflection, and transmission rates (with associated uncertainties) by simulating a large number of neutrons and recording their fates.  
We then plot the variation of these rates as functions of slab thickness for each material.

In [ ]:
def random_walks(slab, counts):
    return [ random_walk(slab) for _ in range(counts)]

def count_fates(slab, counts):
    return Counter([walk[0] for walk in random_walks(slab, counts)])

def binomial_standard_error(probability, trials):
    return np.sqrt(probability * (1 - probability) / trials)

In [ ]:

def plot_width_vs_fates(trials, steps, materials):
    for material, max_width in materials:
        widths = np.linspace(0.000001, max_width, steps)
        data = { }
        for width in widths:
            fates = count_fates(Slab(material, width), trials)
            for fate, count in fates.items():
                probability = count / trials
                error = trials * binomial_standard_error(probability, trials)
                if fate not in data:
                    data[fate] = []
                
                data[fate].append((width, count, error))
       
        fig = go.Figure()

        def exponential_func(x, attenuation_length):
             return trials * np.exp(-x / attenuation_length)
        
        transmitted_widths, transmitted_counts, transmitted_errors = zip(*data["Transmitted"])
        popt, pcov = curve_fit(exponential_func, transmitted_widths, transmitted_counts)

        attenuation_length = popt[0]
        attenuation_length_error = np.sqrt(pcov[0][0])

        theoretical_x = np.linspace(0, max_width, 1000)
        theoretical_y_curve_fit = exponential_func(theoretical_x, attenuation_length)
        theoretical_trace_curve_fit = go.Scatter(
            x=theoretical_x,
            y=theoretical_y_curve_fit,
            mode='lines',
            name=f"Fitted survival function, {material.name}"
        )
        
        fig.add_trace(theoretical_trace_curve_fit)

        for fate_name in data:
            widths, counts, errors = zip(*data[fate_name])
            fig.add_trace(go.Scatter(
                x=widths,
                y=counts,
                mode='markers',
                name=f"{material.name} {fate_name}",
                error_y=dict(
                    type='data',
                    array=errors,
                    visible=True
                )
            ))

        fig.add_trace(go.Scatter(
            x=[attenuation_length, attenuation_length],
            y=[0, trials],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False
        ))

        fig.add_annotation(
            x=attenuation_length,
            y=trials * 0.8,
            text=f"Attenuation length: {attenuation_length:.2f} ± {attenuation_length_error:.2f}",
            showarrow=True,
            arrowhead=2,
            ax=-110,
            ay=-40
        )

        fig.update_layout(
            title=f"Survivability for various widths of {material.name}",
            xaxis_title="Width (cm)",
            yaxis_title="Surviving neutrons",
            width=plot_width,
            height=plot_height,
            legend=dict(
                x=1.05,
                y=1,
                xanchor="left",
                yanchor="top"
            ),
            margin=dict(r=150)
        )

        fig.show()

plot_width_vs_fates(trials=1000, steps=30, materials=[(water, 1), (lead, 10), (graphite, 10)])

While the theoretical model assumes that neutron transmission through a material decreases exponentially with thickness, our simulation reveals a subtle deviation from this idealised behaviour. When plotting the number of transmitted particles as a function of material thickness, the curve fit deviates slightly from a perfect exponential, particularly at larger thicknesses. This discrepancy arises due to the physical structure of the simulation: neutrons start their path lengths at random distances inside the material (consistent with an exponential distribution), but particles that are reflected or absorbed before completing a full exponential step effectively truncate the distribution. As a result, the ensemble of transmission probabilities across many particles is not a pure exponential function of thickness. This effect is small but observable, and fitting only the early portion of the data — before transmission drops below $1/e$ — yields a closer match to the expected exponential decay and a more accurate attenuation length.


---

## 7. Stretch Yourself: Simulating Transmission Through Two Adjacent Slabs Using the Woodcock Method

In this extension task, we apply the **Woodcock method** (also known as delta-tracking) to simulate neutron transmission through two adjacent slabs composed of different materials.

Each slab has a thickness of $L$, giving a total system thickness of $2L$. Neutrons are fired perpendicularly into the slabs at $x = 0$ and undergo stochastic random walks based on the Woodcock sampling method.

### Application of the Woodcock Method

First, we compare the **total macroscopic cross-sections** of the two materials and identify the maximum value, $\Sigma_{\text{max}}$. This value is used for sampling step lengths throughout the entire simulation, ensuring uniform handling of different materials.

At each step:
- A random step length is drawn from an exponential distribution based on $\Sigma_{\text{max}}$.
- The neutron moves along a randomly generated isotropic direction.

After each step:
- If the neutron is currently in the **lower cross-section material**, we apply a **rejection sampling check**:
  - A uniform random number between 0 and 1 is generated.
  - If the random number is **less than** the **acceptance probability** (defined as $\Sigma_{\text{material}} / \Sigma_{\text{max}}$), the interaction is considered **real**.
    - In this case, we proceed to determine whether the neutron is absorbed, transmitted, or reflected.
  - If the random number is **greater than** the acceptance probability, the interaction is treated as a **virtual collision**.
    - No physical interaction (absorption or scattering) occurs.
    - The neutron simply continues with a new random step.

- If the neutron is in the **higher cross-section material** (i.e., $\Sigma = \Sigma_{\text{max}}$), every interaction is automatically accepted without rejection sampling.

Boundary conditions are handled as follows:
- If the neutron moves beyond $x > 2L$, it is considered **transmitted**.
- If it moves back through $x < 0$, it is considered **reflected**.
- If it undergoes a real collision inside the slab, absorption or scattering is determined based on the material properties.

This approach allows efficient and unbiased simulation of neutron transport through inhomogeneous materials with different cross-sections.

In [ ]:
def random_walk_woodcock(slabs):
    position = (0, 0, 0)
    direction = (1, 0, 0)
    path = [position]
    max_cross_section = max(slab.material.total_cross_section for slab in slabs)
    total_width = sum(slab.width for slab in slabs)

    def get_slab_map():
        x_start = 0
        for slab in slabs:
            x_end = x_start + slab.width
            yield (x_start, x_end, slab.material, slab.material.total_cross_section / max_cross_section)
            x_start = x_end

    def get_material_at(x):
        return next(((m, p) for start, end, m, p in slab_positions if start <= x < end), None)

    slab_positions = list(get_slab_map())

    while True:

        step_size = random_uniform_path_length(max_cross_section)
        x = position[0] + step_size * direction[0]
        y = position[1] + step_size * direction[1]
        z = position[2] + step_size * direction[2]
        position = (x, y, z)

        if x < 0:
            path.append(position)
            return "Reflected", path
        if x > total_width:
            path.append(position)
            return "Transmitted", path

        material, p_real = get_material_at(x)
        if material is None:
            raise ValueError("Position out of bounds")

        if np.random.uniform(0, 1) > p_real:
            continue
        else:
            if np.random.uniform(0, 1) < material.absorption_probability:
                path.append(position)
                return "Absorbed", path
            else:
                direction = generate_random_unit_vector()
                path.append(position)

In [ ]:
trials = 3000
counts = Counter([walk[0] for walk in [random_walk_woodcock([Slab(graphite, 10), Slab(lead, 10)]) for _ in range(trials)]])

markdown = "Fate | Count | Rate (%) \n"
markdown += "--- | --- | --- \n"
for fate, count in counts.items():
    rate = count / trials * 100
    markdown += f"{fate} | {count} | {rate:.2f} \n"
markdown += "\n\n"
print_markdown(markdown)


### Validation of the Woodcock Method Implementation

To validate the accuracy of the Woodcock method, we employed several checks:

- **Homogeneous Material Validation:**  
  We first simulated neutron transport through a homogeneous slab using the Woodcock method and compared the results to a standard random walk without Woodcock sampling. The transmission, absorption, and reflection rates showed excellent agreement, confirming correct basic implementation.

- **Step Length Distribution Check:**  
  We plotted histograms of sampled step lengths to verify that they followed the expected exponential distribution governed by $\Sigma_{\text{max}}$.

- **Acceptance Rate Monitoring:**  
  The acceptance probabilities and rejection rates were monitored during simulations to ensure that the sampling logic correctly reflected the cross-section ratios.

- **Material Boundary Handling:**  
  Additional tests were conducted to confirm that neutrons crossing the slab interface correctly switched material properties and updated acceptance probabilities appropriately.

Through these validation steps, we confirmed that the Woodcock method was implemented correctly and reliably simulated neutron transport through a two-slab system.

### 7.1
Homogeneous Material Validation

7.2

In [ ]:
def sample_woodcock_step_lengths(num_samples, sigma_max):
    return -np.log(np.random.uniform(0, 1, size=num_samples)) / sigma_max

sigma_max = max(material.total_cross_section for material in materials)
num_samples = 10000
steps = sample_woodcock_step_lengths(num_samples, sigma_max)

sorted_steps = np.sort(steps)
ranks = np.arange(num_samples, 0, -1)

log_ranks = np.log(ranks)
fit, cov = np.polyfit(sorted_steps, log_ranks, 1, cov=True)
slope, intercept = fit
slope_uncertainty = np.sqrt(cov[0, 0])

fig = go.Figure()
fig.add_trace(go.Scatter(x=sorted_steps, y=ranks, mode='markers', name='Ranked survival'))
fig.add_trace(go.Scatter(x=sorted_steps, y=np.exp(intercept + slope * sorted_steps), mode='lines', name='Fitted exponential'))

fig.update_layout(
    title="Ranked Method: Woodcock Step Lengths",
    xaxis_title="Step length (cm)",
    yaxis_title="Number of neutrons surviving",
    width=800,
    height=500
)
fig.show()

print(f"Fitted slope: {slope:.4f} ± {slope_uncertainty:.4f}")
print(f"Expected slope: {-sigma_max:.4f}")
print(f"Estimated mean free path: {-1 / slope:.4f} cm (expected: {1 / sigma_max:.4f} cm)")

reduced_chi_square = calculate_reduced_chi_square(ranks, np.exp(intercept + slope * sorted_steps))
print(f"Reduced chi-square: {reduced_chi_square:.4f}")

In [ ]:
end = time.time()
print(f"Execution time: {end - start:.2f} seconds")